In [76]:
import re, os, sys
from functools import reduce
from itertools import combinations as comb, permutations as perm, combinations_with_replacement as combr
from operator import itemgetter
from pprint import pprint
from math import *
from collections import defaultdict
import networkx as nx
import numpy as np

def dprint(d):
    pprint(dict(d))
def lmap(*args, **kwargs):
    return list(map(*args, **kwargs))
def lmapi(*args, **kwargs):
    return list(map(int, *args, **kwargs))
def enum(*args, **kwargs):
    return enumerate(*args, **kwargs)

In [77]:
input_fname = "../inputs/day22-demo.in"

In [121]:
input_fname = "../inputs/day22.in"

In [122]:
def reread():
    lines = []
    with open(input_fname) as ifile:
        def r(): return ifile.readline().strip()
        def rs(): return ifile.readline().strip().split()
        def ri(): return int(ifile.readline().strip())
        def ris(): return list(map(int, ifile.readline().strip().split()))

        rows = []
        maxwidth = 0
        # read map
        for line in ifile:
            if not line.strip(): break
            row = [" .#".index(c) for c in line.rstrip()]
            rows.append(row)
            maxwidth = max(maxwidth, len(row))

        ar = np.zeros((len(rows), maxwidth), dtype=int)
        for i, row in enum(rows):
            ar[i, :len(row)]  = row
        # directions
        path = r()
        path = path.replace("R", " R ").replace("L", " L ").split()
    
    return ar, path

ar, path = reread()
# ar[:10], path[:10]

In [81]:
# debug cube in layout of large input
ar = np.repeat(np.repeat(SIDENUMS, DIM, axis=1), DIM, axis=0)

## Part 1

In [123]:
rowlos = []
rowhis = []
for row in ar:
    idx = np.nonzero(row > 0)
    rowlos.append(np.min(idx))
    rowhis.append(np.max(idx))
# rowlos, rowhis

In [124]:
collos = []
colhis = []
for col in ar.T:
    idx = np.nonzero(col > 0)
    collos.append(np.min(idx))
    colhis.append(np.max(idx))
# collos, colhis

In [125]:
def i2c(cn):
    return int(cn.real), int(cn.imag)
def c2i(x, y):
    return x+1j*y

In [126]:
def travel(strip, relpos, dist):
    # print("travel got", strip, relpos, dist)
    if np.max(strip) == 1:  # no wall
        newpos = relpos + dist
        # print(f"no wall")#, {pos}->{newpos}")
    else:  # some wall
        rem = np.hstack((strip, strip))[relpos:relpos+len(strip)]
        # print(np.max(strip), rem, (rem == 2).nonzero())
        walldist = (rem == 2).nonzero()[0][0]
        if dist >= walldist:
            # hit wall
            newpos = relpos + walldist - 1
            # print(f"hit wall")#, {pos}->{newpos}")
        else:
            # no hit
            newpos = relpos + dist
            # print(f"free move")#, {pos}->{newpos}")
    return newpos % len(strip)

In [127]:
pos = c2i(rowlos[0], collos[rowlos[0]])
face = 1+0j
for move in path:
    if move == "R":
        face *= 1j
        # print("turning right", face)
    elif move == "L":
        face *= -1j
        # print("turning left", face)
    else:
        dist = int(move)
        x, y = i2c(pos)
        if face == 1:  # right
            row = ar[y][ar[y]>0]
            lo = rowlos[y]
            xrel = x - lo
            newx = lo + travel(row, xrel, dist)
            newpos = c2i(newx, y)
        elif face == -1:  # left
            row = ar[y][ar[y]>0][::-1]
            hi = rowhis[y]
            xrel = hi - x
            newx = hi - travel(row, xrel, dist)
            newpos = c2i(newx, y)
        elif face == 1j:  # down
            col = ar[:,x][ar[:,x]>0]
            lo = collos[x]
            yrel = y - lo
            newy = lo + travel(col, yrel, dist)
            newpos = c2i(x, newy)
        elif face == -1j:  # up
            col = ar[:,x][ar[:,x]>0][::-1]
            hi = colhis[x]
            yrel = hi - y
            newy = hi - travel(col, yrel, dist)
            newpos = c2i(x, newy)
        else:
            raise RuntimeError(f"shouldn't reach here, face: {face}")
        # print(f"updating pos, {i2c(pos)}->{i2c(newpos)}")
        pos = newpos
    # break
print("final:", pos)
password = 0
password += (pos.imag + 1)*1000
password += (pos.real + 1)*4
password += [1, -1j, -1, 1j].index(face)
print("password:", int(password))

final: (96+66j)
password: 67390


## Part 2

# Try next

> probably need to encode 3 bits of information and complex number isn't enough
> - next side
> - flip or not (`DIM - 1 - yin`)
> - next face on next side (includes `reverse` or not, only `-1` and `-1j` are reverse)

In [22]:
# small params
DIM = 4
SIDENUMS = np.array([
    [0, 0, 1, 0],
    [2, 3, 4, 0],
    [0, 0, 5, 6]
])

NBRS = {
    -1: {  # left
        1:(3,1j), 2:(-6,-1j), 3:(2,-1),
        4:(3,-1), 5:(-3,-1j), 6:(5,-1)
    },
    1: {  # right
        1:(-6,-1), 2:(3,1), 3:(4,1),
        4:(-6,1j), 5:(6,1), 6:(-1,-1)
    },
    -1j: {  # up
        1:(-2,1j), 2:(-1,1j), 3:(1,1),
        4:(1,-1j), 5:(4,-1j), 6:(-4,-1)
    },
    1j: {  # down
        1:(4,1j), 2:(-5,-1j), 3:(-5,1),
        4:(5,1j), 5:(-2,-1j), 6:(-2,1)
    },
}

In [130]:
# large params
DIM = 50
SIDENUMS = np.array([
    [0, 2, 1],
    [0, 3, 0],
    [5, 4, 0],
    [6, 0, 0]
])

NBRS = {
    -1: {  # left
        1:(2,-1), 2:(-5,1), 3:(5,1j),
        4:(5,-1), 5:(-2,1), 6:(2,1j)
    },
    1: {  # right
        1:(-4,-1), 2:(1,1), 3:(1,-1j),
        4:(-1,-1), 5:(4,1), 6:(4,-1j)
    },
    -1j: {  # up
        1:(6,-1j), 2:(6,1), 3:(2,-1j),
        4:(3,-1j), 5:(3,1), 6:(5,-1j)
    },
    1j: {  # down
        1:(3,-1), 2:(3,1j), 3:(4,1j),
        4:(6,-1), 5:(6,1j), 6:(1,1j)
    },
}

In [131]:
def find_side(pos):
    # print(i2c(pos), end="->")
    pos /= DIM
    # print(i2c(pos), end="\t")
    x, y = i2c(pos)
    # print("x", x, "y", y, "\t", sides[y,x])
    return SIDENUMS[y,x]

In [ ]:
# debug cube
nonzeros = list(zip(*ar.nonzero()))
ctr = 1
for y, x in nonzeros:
    ar[y,x] = ctr
    ctr += 1
ar

```
                            +-------------+
                            |  1  2  3  4 |           
                            |  5  6  7  8 |           
                            |  9 10 11 12 |           
                            | 13 14 15 16 |           
+-------------+-------------+-------------+
| 17 18 19 20 | 21 22 23 24 | 25 26 27 28 |           
| 29 30 31 32 | 33 34 35 36 | 37 38 39 40 |           
| 41 42 43 44 | 45 46 47 48 | 49 50 51 52 |           
| 53 54 55 56 | 57 58 59 60 | 61 62 63 64 |
+-------------+-------------+-------------+-------------+         
                            | 65 66 67 68 | 69 70 71 72 |
                            | 73 74 75 76 | 77 78 79 80 |
                            | 81 82 83 84 | 85 86 87 88 |
                            | 89 90 91 92 | 93 94 95 96 |
                            +-------------+-------------+
```

In [132]:
SIDES = dict()
POSES = dict()
for sy, srow in enum(SIDENUMS):
    for sx, sidenum in enum(srow):
        if sidenum:
            SIDES[sidenum] = ar[sy*DIM:(sy+1)*DIM, sx*DIM:(sx+1)*DIM]
            ix, iy = np.meshgrid(
                np.arange(sx*DIM, (sx+1)*DIM), np.arange(sy*DIM, (sy+1)*DIM)
            )
            POSES[sidenum] = ix+iy*1j
            # print("extracted side", sidenum)
            # print(ar[sy*DIM:(sy+1)*DIM, sx*DIM:(sx+1)*DIM])

In [133]:
def collect_strips(ar, pos, face):
    def isneg(cn):
        return cn.real < 0 or cn.imag < 0
    x, y = i2c(pos)
    xin, yin = x%DIM, y%DIM
    # offset = yin if face.real == 0 else xin
    side = find_side(pos)
    assert side in range(1, 7), f"invalid pos {pos}, not on any side"
    
    strips = []
    poslist = []
    facelist = []
    for i in range(4):
        facelist.extend([face]*DIM)
        if face.imag == 0:  # horizontal strip
            strip = SIDES[side][yin]
            poses = POSES[side][yin]
        else:  # vertical strip
            strip = SIDES[side][:,xin]
            poses = POSES[side][:,xin]
        if isneg(face): 
            strip = strip[::-1]
            poses = poses[::-1]
        # print("strip", i, strip)
        strips.append(strip)
        poslist.append(poses)
        # print()
        
        # prepare for next strip
        nxt, nxtface = NBRS[face][side]
        nxtside = int(abs(nxt))
        side = nxtside
        turn = ((face*nxtface).imag != 0)
        # print("face", face, "\tnxtface", nxtface, "\tturn", turn)
        face = nxtface
        if nxt < 0:
            yin = (DIM - 1) - yin
            xin = (DIM - 1) - xin
        if turn:
            yin, xin = xin, yin
        # print("next side", nxtside, "\tface", face,
            #  "\txin", xin, "\tyin", yin)
        
    stacposes = np.hstack(poslist)
    offset = (stacposes == pos).nonzero()[0][0]
    retstrips = np.roll(np.hstack(strips), -offset)
    retposes = np.roll(stacposes, -offset)
    retfaces = np.roll(facelist, -offset)
    return retstrips, retposes, retfaces

In [107]:
# collect_strips(ar, 8+0j, 1)
#1,2,3,4,96,95,94,93,92,91,90,89,57,45,33,21
# collect_strips(ar, 15+11j, -1)
#96,95,94,93,92,91,90,89,57,45,33,21,1,2,3,4
# collect_strips(ar, 7+5j, 1j)
#24,36,48,60,65,66,67,68,69,70,71,72,16,15,14,13
# collect_strips(ar, 10, 1j)
collect_strips(ar, 7, -1j)

(array([ 4, 93, 94, 95, 96, 80, 72, 64, 56, 48, 44, 40, 36, 28, 20, 12]),
 array([7. +0.j, 0.+15.j, 1.+15.j, 2.+15.j, 3.+15.j, 7.+11.j, 7.+10.j,
        7. +9.j, 7. +8.j, 7. +7.j, 7. +6.j, 7. +5.j, 7. +4.j, 7. +3.j,
        7. +2.j, 7. +1.j]),
 array([-0.-1.j,  1.+0.j,  1.+0.j,  1.+0.j,  1.+0.j, -0.-1.j, -0.-1.j,
        -0.-1.j, -0.-1.j, -0.-1.j, -0.-1.j, -0.-1.j, -0.-1.j, -0.-1.j,
        -0.-1.j, -0.-1.j]))

In [136]:
def cube_travel(strip, dist):
    # print("travel got", strip, dist)
    if np.max(strip) == 1:  # no wall
        newpos = dist
        # print(f"no wall")#, {pos}->{newpos}")
    else:  # some wall
        rem = np.hstack((strip, strip))[:len(strip)]
        # print(np.max(strip), rem, (rem == 2).nonzero())
        walldist = (rem == 2).nonzero()[0][0]
        if dist >= walldist:
            # hit wall
            newpos = walldist - 1
            # print(f"hit wall")#, {pos}->{newpos}")
        else:
            # no hit
            newpos = dist
            # print(f"free move")#, {pos}->{newpos}")
    return newpos % len(strip)

In [137]:
pos = c2i(rowlos[0], collos[rowlos[0]])
face = 1+0j
for move in path:
    if move == "R":
        face *= 1j
        # print("turning right", face)
    elif move == "L":
        face *= -1j
        # print("turning left", face)
    else:
        dist = int(move)
        x, y = i2c(pos)
        # print("passing", pos, face)
        strip, poslist, facelist = collect_strips(ar, pos, face)
        # print("strip", strip)
        # print(poslist)
        newidx = cube_travel(strip, dist)
        newpos = poslist[newidx]
        newface = facelist[newidx]
        # print(f"updating pos, {i2c(pos)}->{i2c(newpos)}")
        # print(f"updating face, {face}->{newface}")
        # print()
        pos = newpos
        face = newface
    # break
print("final:", pos)
password = 0
password += (pos.imag + 1)*1000
password += (pos.real + 1)*4
password += [1, 1j, -1, -1j].index(face)
print("password:", int(password))

final: (71+94j)
password: 95291


- `110080` - too high
- `110083` - too high
- `95291` - just right